In [1]:
from abc import ABCMeta, abstractmethod
from queue import PriorityQueue
import random
import math

# Hill-Climbing Search implementation

<img src="hill.png" width="400"/>

In [7]:
class HillClimbing():
    """
    Hill-Climbing Search implementation

    Methods
    -------
    search(state)
        Runs Hill-Climbing Search starting from a given start state, and returns the solution and the number of expanded states.
    successor(state)
        Finds the list of successors for a given state.
    goal_test(state)
        Checks if the current state is a goal state.
    """

    __metaclass__ = ABCMeta

    def search(self, state):
        """
        Runs Hill-Climbing Search starting from a given start state, and returns the solution and the number of expanded states.

        Parameters
        ----------
        state
            A tuple describing a unique world configuration.

        Returns
        -------
        list
            A list of actions that must be applied in sequence to the start state to reach the goal state, or None if no solution was found.
        int
            The number of states expanded during the search.
        """

        num_expanded_states = 0                                   # number of states expanded during the search

        all_time_best = 0
        plateau_count = 0
        max_iterations = 100

        states_to_expand = PriorityQueue()                        # stack of states to be expanded
        #states_to_expand.put((math.inf, state))
        states_to_expand.put((math.inf, math.inf, state))
        #while not states_to_expand.empty():
        while not states_to_expand.empty() and plateau_count < max_iterations:
            #best_cost, state = states_to_expand.get()             # get next state to be expanded
            best_cost, _, state = states_to_expand.get()

            if best_cost < all_time_best:
                all_time_best = best_cost
                plateau_count = 0
            else:
                plateau_count += 1

            num_expanded_states += 1                              # add current state to the solution

            if self.goal_test(state):                             # if current state is a goal state, return solution
                return state, num_expanded_states

            states_to_expand = PriorityQueue()
            for child, heur_cost in self.successor(state):        # add successors to the stack of states to be expanded
                #if heur_cost < best_cost:
                #    states_to_expand.put((heur_cost, child))
                if heur_cost <= best_cost:
                    states_to_expand.put((heur_cost, random.uniform(0, 1), child))

        return None, num_expanded_states                          # if no solution is found, return None

    @abstractmethod
    def successor(self, state):
        """
        Finds the list of successors for a given state.

        Parameters
        ----------
        state
            A tuple describing a unique world configuration.

        Returns
        -------
        list
            A list of pairs (action,state) with all states that can be reached from the given state with a single action.
        """
        pass

    @abstractmethod
    def goal_test(self, state):
        """
        Checks if the current state is a goal state.

        Parameters
        ----------
        state
            A tuple describing a unique world configuration.

        Returns
        -------
        bool
             True if the given state is a goal state, and False otherwise.
        """
        pass

# 8-queens

- Puzzle with chess queens in an $8\times8$ grid. The goal is to place 8 queens so that no queen attacks another.

<img src="8queens.png" width="300"/>

- **Heuristic function**: number of queen pairs attacking each other.

In [8]:
class KQueens(HillClimbing):
    """
    8-queens solution using Hill-Climbing Search.

    Methods
    -------
    show()
        Visualize the current state.
    move(action)
        Apply an action to the current state.
    successor(state)
        Finds the list of successors for a given state.
    goal_test(state)
        Checks if the current state is a goal state.
    """

    def __init__(self, state=None, k=8):
        """
        Parameters
        ----------
        state
            A tuple describing a unique world configuration.
        k
            An integer defining the size of the board and the number of queens.
        """
        self.k = k
        if state is not None:
            self.state = state
        else:
            self.state = self.__get_random_state()

    def __get_random_state(self):
        """
        Generates a random puzzle configuration.

        Return
        ----------
        tuple
            A tuple describing a unique puzzle configuration.
        """
        return tuple(random.randrange(self.k) for i in range(self.k))

    def show(self):
        """
        Prints the current state.
        """
        print('╔'+'╦'.join(['═══']*self.k)+'╗')
        for i in range(self.k):
            print('║', end=' ')
            for j in range(self.k):
                if self.state[j] == i:
                    print('W', end=' ')
                else:
                    print(' ', end=' ')
                print('║', end=' ')
            print()
            if i < self.k-1:
                print('╠'+'╬'.join(['═══']*self.k)+'╣')
        print('╚'+'╩'.join(['═══']*self.k)+'╝')

    def move(self, action):
        """
        Move the queen to a different row in the same column.

        Parameters
        ----------
        action
            A pair of integers defining column and row, respectively.
        """
        a, b = action
        self.state[a] = b

    def heuristic_function(self, state):
        """
        Computes the heuristic function for a given state.

        Parameters
        ----------
        state
            A tuple describing a unique puzzle configuration.

        Returns
        -------
        list
            An integer representing the number of queen pairs that can attack each other.
        """
        cost = 0
        for i in range(self.k):
            for j in range(i+1,self.k):
                if state[i] == state[j] or j-i == abs(state[j]-state[i]):
                    cost += 1
        return cost

    def successor(self, state):
        """
        Finds the list of successors for a given state.

        Parameters
        ----------
        state
            A tuple describing a unique puzzle configuration.

        Returns
        -------
        list
            A list of pairs (state,heuristic) with all states that can be reached from the given states with a single move.
        """
        successors = []
        for i in range(self.k):
            child = list(state)
            for j in range(self.k):
                if j != state[i]:
                    child[i] = j
                    successors.append((tuple(child), self.heuristic_function(child)))
        return successors

    def goal_test(self, state):
        """
        Checks if no queen can attack another.

        Parameters
        ----------
        state
            A tuple describing a unique puzzle configuration.

        Returns
        -------
        bool
             True if the given state is a goal state, and False otherwise.
        """

        return self.heuristic_function(state) == 0

In [11]:
puzzle = KQueens()
print('Start state:')
puzzle.show()

state, num_states = puzzle.search(puzzle.state)
if state is not None:
    puzzle = KQueens(state=state)
    print('Found a solution after expanding {} states!'.format(num_states))
    puzzle.show()
else:
    print('Could not find a solution after expanding {} states!'.format(num_states))

Start state:
╔═══╦═══╦═══╦═══╦═══╦═══╦═══╦═══╗
║   ║   ║ W ║   ║ W ║   ║   ║   ║ 
╠═══╬═══╬═══╬═══╬═══╬═══╬═══╬═══╣
║   ║   ║   ║   ║   ║   ║ W ║   ║ 
╠═══╬═══╬═══╬═══╬═══╬═══╬═══╬═══╣
║ W ║   ║   ║   ║   ║   ║   ║   ║ 
╠═══╬═══╬═══╬═══╬═══╬═══╬═══╬═══╣
║   ║ W ║   ║   ║   ║ W ║   ║   ║ 
╠═══╬═══╬═══╬═══╬═══╬═══╬═══╬═══╣
║   ║   ║   ║   ║   ║   ║   ║   ║ 
╠═══╬═══╬═══╬═══╬═══╬═══╬═══╬═══╣
║   ║   ║   ║   ║   ║   ║   ║   ║ 
╠═══╬═══╬═══╬═══╬═══╬═══╬═══╬═══╣
║   ║   ║   ║ W ║   ║   ║   ║ W ║ 
╠═══╬═══╬═══╬═══╬═══╬═══╬═══╬═══╣
║   ║   ║   ║   ║   ║   ║   ║   ║ 
╚═══╩═══╩═══╩═══╩═══╩═══╩═══╩═══╝
Found a solution after expanding 28 states!
╔═══╦═══╦═══╦═══╦═══╦═══╦═══╦═══╗
║   ║   ║   ║ W ║   ║   ║   ║   ║ 
╠═══╬═══╬═══╬═══╬═══╬═══╬═══╬═══╣
║   ║   ║   ║   ║   ║   ║ W ║   ║ 
╠═══╬═══╬═══╬═══╬═══╬═══╬═══╬═══╣
║   ║   ║ W ║   ║   ║   ║   ║   ║ 
╠═══╬═══╬═══╬═══╬═══╬═══╬═══╬═══╣
║   ║   ║   ║   ║   ║   ║   ║ W ║ 
╠═══╬═══╬═══╬═══╬═══╬═══╬═══╬═══╣
║   ║ W ║   ║   ║   ║   ║   ║   ║ 
╠═══╬═══╬═══

In [12]:
N = 100

num_success = 0
cost_success = 0
num_failure = 0
cost_failure = 0

for i in range(N):
    puzzle = KQueens()
    state, num_states = puzzle.search(puzzle.state)
    if state is not None:
        num_success += 1
        cost_success += num_states
    else:
        num_failure += 1
        cost_failure += num_states

print('Success rate: {:.2f}%'.format(100.0*num_success/N))
print('Average success cost: {:.2f}'.format(cost_success/num_success))
print()
print('Failure rate: {:.2f}%'.format(100.0*num_failure/N))
print('Average failure cost: {:.2f}'.format(cost_failure/num_failure))

Success rate: 95.00%
Average success cost: 21.11

Failure rate: 5.00%
Average failure cost: 100.00
